In [ ]:
import pandas as pd
import xarray as xr
import holoviews as hv
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

hv.extension('bokeh')

In [ ]:
their_model = pd.read_excel('/Volumes/LtgSSD/arm-ccn-fix/modeloutput.xlsx', skiprows=1).set_index('Timestamp (UTC)', drop=True)

In [ ]:
their_zero_point_one = their_model['CCN @ SS = 0.1% (cm-3)']
their_one = their_model['CCN @ SS = 1% (cm-3)']

In [ ]:
aos_obs = xr.open_mfdataset('/Volumes/LtgSSD/arm-ccn-avg/*.nc')
aos_obs.supersaturation_calculated.plot.hist(bins=100)
plt.xlabel('Supersaturation (%)')
plt.ylabel('Frequency')
plt.title('Supersaturation Distribution in AOS CCN Data')
aos_zero_point_one = aos_obs.isel(time=((aos_obs.supersaturation_calculated <= 0.15)).compute())
aos_zero_point_four = aos_obs.isel(time=((aos_obs.supersaturation_calculated >= 0.4) & (aos_obs.supersaturation_calculated <= 0.6)).compute())
aos_zero_point_six = aos_obs.isel(time=((aos_obs.supersaturation_calculated >= 0.6) & (aos_obs.supersaturation_calculated <= 0.8)).compute())
aos_one = aos_obs.isel(time=((aos_obs.supersaturation_calculated >= 0.8) & (aos_obs.supersaturation_calculated <= 1.05)).compute())

In [ ]:
my_model = pd.read_csv('/Volumes/LtgSSD/arm-ccn-fix/sam_pyrcel_out.csv', parse_dates=['timestamp']).set_index('timestamp').drop(columns=['Unnamed: 0'])
my_zero_point_one = my_model['0.1SS']
my_zero_point_four = my_model['0.4SS']
my_zero_point_six = my_model['0.6SS']
my_one = my_model['1.0SS']

In [ ]:
wrong_curve = hv.Scatter((aos_zero_point_one.time, aos_zero_point_one.N_CCN), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='AOS CCN Instrument').opts(title='CCN Timeseries, 0.1%SS', width=800, height=400, color='red', line_width=2)
their_curve = hv.Scatter((their_model.index, their_zero_point_one), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='Petters Pyrcel').opts(width=800, height=400, color='blue', line_width=2)
my_curve = hv.Scatter((my_model.index, my_zero_point_one.values), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='Sam Pyrcel').opts(width=800, height=400, color='green', line_width=2)

wrong_curve * their_curve * my_curve

In [ ]:
wrong_curve = hv.Scatter((aos_zero_point_four.time, aos_zero_point_four.N_CCN), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='AOS CCN Instrument').opts(title='CCN Timeseries, 0.4%SS', width=800, height=400, color='red', line_width=2)
my_curve = hv.Scatter((my_model.index, my_zero_point_four.values), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='Sam Pyrcel').opts(width=800, height=400, color='green', line_width=2)

wrong_curve * my_curve

In [ ]:
wrong_curve = hv.Scatter((aos_zero_point_six.time, aos_zero_point_six.N_CCN), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='AOS CCN Instrument').opts(title='CCN Timeseries, 0.6%SS', width=800, height=400, color='red', line_width=2)
my_curve = hv.Scatter((my_model.index, my_zero_point_six.values), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='Sam Pyrcel').opts(width=800, height=400, color='green', line_width=2)

wrong_curve * my_curve

In [ ]:
wrong_curve = hv.Scatter((aos_one.time, aos_one.N_CCN), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='AOS CCN Instrument').opts(title='CCN Timeseries, 1%SS', width=800, height=400, color='red', line_width=2)
their_curve = hv.Scatter((their_model.index, their_one), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='Petters Pyrcel').opts(width=800, height=400, color='blue', line_width=2)
my_curve = hv.Scatter((my_model.index, my_one.values), kdims=['Time'], vdims=['N_CCN (cm^-3)'], label='Sam Pyrcel').opts(width=800, height=400, color='green', line_width=2)

wrong_curve * their_curve * my_curve

In [ ]:
hourly_obs_point_one = aos_zero_point_one.resample(time='1H').mean()
hourly_obs_point_four = aos_zero_point_four.resample(time='1H').mean()
hourly_obs_point_six = aos_zero_point_six.resample(time='1H').mean()
hourly_obs_one = aos_one.resample(time='1H').mean()

In [ ]:
hourly_obs_matching_point_one = hourly_obs_point_one.isel(time=hourly_obs_point_one.time.isin(my_zero_point_one.index))
hourly_obs_matching_point_four = hourly_obs_point_four.isel(time=hourly_obs_point_four.time.isin(my_zero_point_four.index))
hourly_obs_matching_point_six = hourly_obs_point_six.isel(time=hourly_obs_point_six.time.isin(my_zero_point_six.index))
hourly_obs_matching_one = hourly_obs_one.isel(time=hourly_obs_one.time.isin(my_one.index))

In [ ]:
my_model_matching = my_model.loc[hourly_obs_matching_one.time]
my_matching_point_one = my_zero_point_one.loc[hourly_obs_matching_point_one.time]
my_matching_point_four = my_zero_point_four.loc[hourly_obs_matching_point_four.time]
my_matching_point_six = my_zero_point_six.loc[hourly_obs_matching_point_six.time]
my_matching_one = my_one.loc[hourly_obs_matching_one.time]
their_model_matching = their_model.loc[hourly_obs_matching_one.time]
their_matching_point_one = their_zero_point_one.loc[hourly_obs_matching_point_one.time]
their_matching_one = their_one.loc[hourly_obs_matching_one.time]

In [ ]:
all_nonnan = (~np.isnan(their_matching_point_one.values)) & (~np.isnan(hourly_obs_matching_point_one.N_CCN.data))
res = linregress(their_matching_point_one.values[all_nonnan], hourly_obs_matching_point_one.N_CCN.data[all_nonnan])
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr
hv.Scatter((their_matching_point_one.values[all_nonnan], hourly_obs_matching_point_one.N_CCN.data[all_nonnan])).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs Petters\' pyrcel model value at <0.15%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')

In [ ]:
all_nonnan = (~np.isnan(my_matching_point_one.values)) & (~np.isnan(hourly_obs_matching_point_one.N_CCN.data))
res = linregress(my_matching_point_one.values[all_nonnan], hourly_obs_matching_point_one.N_CCN.data[all_nonnan])
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr
hv.Scatter((my_matching_point_one.values[all_nonnan], hourly_obs_matching_point_one.N_CCN.data[all_nonnan])).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs Sam\'s pyrcel model value at <0.15%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')

In [ ]:
all_nonnan = (~np.isnan(my_matching_point_four.values)) & (~np.isnan(hourly_obs_matching_point_four.N_CCN.data))
res = linregress(my_matching_point_four.values[all_nonnan], hourly_obs_matching_point_four.N_CCN.data[all_nonnan])
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr
hv.Scatter((my_matching_point_four.values[all_nonnan], hourly_obs_matching_point_four.N_CCN.data[all_nonnan])).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs Sam\'s pyrcel model value at 0.4-0.6%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')

In [ ]:
all_nonnan = (~np.isnan(my_matching_point_six.values)) & (~np.isnan(hourly_obs_matching_point_six.N_CCN.data))
res = linregress(my_matching_point_six.values[all_nonnan], hourly_obs_matching_point_six.N_CCN.data[all_nonnan])
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr
hv.Scatter((my_matching_point_six.values[all_nonnan], hourly_obs_matching_point_six.N_CCN.data[all_nonnan])).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs Sam\'s pyrcel model value at 0.6-0.8%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')

In [ ]:
all_nonnan = (~np.isnan(my_matching_one.values)) & (~np.isnan(hourly_obs_matching_one.N_CCN.data))
res = linregress(my_matching_one.values[all_nonnan], hourly_obs_matching_one.N_CCN.data[all_nonnan])
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr
hv.Scatter((my_matching_one.values[all_nonnan], hourly_obs_matching_one.N_CCN.data[all_nonnan])).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs Sam\'s pyrcel model value at 0.8-1.05%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')

In [ ]:
all_nonnan = (~np.isnan(their_matching_one.values)) & (~np.isnan(hourly_obs_matching_one.N_CCN.data))
res = linregress(their_matching_one.values[all_nonnan], hourly_obs_matching_one.N_CCN.data[all_nonnan])
m = res.slope
b = res.intercept
r = res.rvalue
p = res.pvalue
stderr = res.stderr
int_stderr = res.intercept_stderr
hv.Scatter((their_matching_one.values[all_nonnan], hourly_obs_matching_one.N_CCN.data[all_nonnan])).opts(width=800, height=400, color='green', xlabel='pyrcel model', ylabel='AOS CCN', title=f'Averaged Hourly AOS CCN observation vs Petters\' pyrcel model value at 0.8-1.05%SS\nm = {m:.2f}, b = {b:.2f}, r^2 = {r**2:.2f}, p = {p:.2f}')